In [ ]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re
pd.set_option('display.max_columns', None)

In [ ]:
# location of raw data files
input_path = "/home/jupyter/datasets/raw/"

In [ ]:
#read PROCEDUREEVENTS_MV.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'NOTEEVENTS.csv',dtype = "object")

In [ ]:
df.info(verbose=True, null_counts=True)

In [ ]:
df.head()

In [ ]:
#Dropping rows that have error data
df = df[df['ISERROR']!='1'].reset_index(drop = True)

In [ ]:
# Replacing missing values for STORTIME with CHARTDATE
df.loc[df['STORETIME'].isnull(),'STORETIME']=df['CHARTDATE']
df['STORETIME'] = pd.to_datetime(df['STORETIME'])

In [ ]:
df.head()

In [ ]:
df = df.drop(labels=['CHARTDATE','CHARTTIME','CGID','ISERROR'], axis = 'columns')

In [ ]:
df.head()

In [ ]:
%%time
#Cleaning the data by removing special charecters and masked data
df['TEXT'] = df['TEXT'].map(lambda x: re.sub('\[.*?\]|[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', ' ', x)).copy()

In [ ]:
df.head()

In [ ]:
# Create Notevents
df['noteevents'] = df['CATEGORY']+' '+df['DESCRIPTION']+' '+df['TEXT']

In [ ]:
df = df.drop(labels=['CATEGORY','DESCRIPTION','TEXT'], axis = 'columns')

In [ ]:
# not using a map function as it occupies a lot of memory
from tqdm.auto import tqdm
for i in tqdm(range(len(df))):
    df.at[i, ('noteevents')] = df.at[i, ('noteevents')].split()

In [ ]:
df.head()

In [ ]:
%%time
df['STORETIME'] = df['STORETIME'].map(lambda x: str(x))

In [ ]:
# create output path
mypath_output = "/home/jupyter/datasets/noteevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [ ]:
# export the dataframe to JSON format
df.to_json(mypath_output+"noteevents.json",orient = 'records') 

In [ ]:
df['len'] = df['noteevents'].map(lambda x: len(x))

In [ ]:
df['len'].describe()

In [ ]:
sum(df['len'])

In [ ]:
len(df)